In [1]:

import os
import pandas as pd
import warnings
from tqdm import tqdm

# Filter out warning
warnings.filterwarnings('ignore', category=UserWarning, module='tqdm')

data = "updatedCveDB.csv"

if not os.path.exists(data):
    print(f"[INFO] Downloading data from MITRE...")
    url = "https://cve.mitre.org/data/downloads/allitems.csv"
    df = pd.read_csv(url, skiprows=2, encoding='latin-1', dtype=str)
    total_rows = len(df)
    with tqdm(total=total_rows, desc="Processing data") as pbar:
        df.to_csv(data, index=False)
        pbar.update(total_rows)

[INFO] Downloading data from MITRE...


Processing data: 100%|██████████| 311268/311268 [00:02<00:00, 104743.46it/s]


In [2]:
# Load processed data
df = pd.read_csv(data, usecols=["Name", "Description"])

# Remove rows where description starts with disclaimers
df = df.dropna(subset=['Description'])
df = df[~df['Description'].str.startswith('**')]

# Save the processed data
df.to_csv('updatedCveDB.csv', index=False)
print(f"[INFO] Data saved to updatedCveDB.csv")

# Make another csv with 500 random rows for testing
df.sample(500).to_csv('sample.csv', index=False)

df1 = pd.read_csv('sample.csv')

[INFO] Data saved to updatedCveDB.csv


In [3]:
import pandas as pd
from transformers import pipeline
# Initialize the question-answering pipeline with a pre-trained model
qa_pipeline = pipeline('question-answering', model='deepset/roberta-base-squad2')

# Function to ask questions about the data
def ask_question(question):
    context = ' '.join((df1['CVE'] + ' ' + df1['Description']).tolist())
    result = qa_pipeline(question=question, context=context)
    return result['answer']

# Example usage
question = input("Prompt:")
answer = ask_question(question)
print(f"Question: {question}")
print(f"Answer: {answer}")

ValueError: Arguments can't be understood